In [1]:
import robosuite as suite
from robosuite.wrappers.gym_wrapper import GymWrapper
import gymnasium as gym
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor
import time
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /home/ja/PycharmProjects/robosuite/robosuite/robosuite/scripts/setup_macros.py (macros.py:55)


In [2]:
import os
filename = 'tmp/gym/twoarm_ppo'
cwd = os.getcwd()
new_folder = os.path.join(cwd, filename)
os.makedirs(new_folder, exist_ok=True)

In [3]:
# create environment instance
env = GymWrapper(suite.make(
    env_name="TwoArmPegInHole", # try with other tasks like "Stack" and "Door"
    robots=["UR5e","UR5e"],  # try with other robots like "Sawyer" and "Jaco"
    has_renderer=True,
    has_offscreen_renderer=True,
    use_object_obs=False,                   # don't provide object observations to agent
    use_camera_obs=True,
    camera_names="robot0_eye_in_hand",      # use "agentview" camera for observations
    camera_heights=84,                      # image height
    camera_widths=84,                       # image width
    reward_shaping=True,                    # use a dense reward signal for learning
    horizon = 500,
    control_freq=20,                        # control should happen fast enough so that simulation looks smooth
))
env = Monitor(env, filename)
# ), ['robot0_eye_in_hand_image'])

In [4]:
env.observation_space

Dict('robot0_eef_pos': Box(-1.0, 1.0, (3,), float32), 'robot0_eef_quat': Box(-1.0, 1.0, (4,), float32), 'robot0_eye_in_hand_image': Box(0, 255, (84, 84, 3), uint8), 'robot0_joint_pos_cos': Box(-1.0, 1.0, (6,), float32), 'robot0_joint_pos_sin': Box(-1.0, 1.0, (6,), float32), 'robot0_joint_vel': Box(-1.0, 1.0, (6,), float32), 'robot0_proprio-state': Box(-1.0, 1.0, (25,), float32), 'robot1_eef_pos': Box(-1.0, 1.0, (3,), float32), 'robot1_eef_quat': Box(-1.0, 1.0, (4,), float32), 'robot1_joint_pos_cos': Box(-1.0, 1.0, (6,), float32), 'robot1_joint_pos_sin': Box(-1.0, 1.0, (6,), float32), 'robot1_joint_vel': Box(-1.0, 1.0, (6,), float32), 'robot1_proprio-state': Box(-1.0, 1.0, (25,), float32))

In [5]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), "timesteps")
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print(f"Num timesteps: {self.num_timesteps}")
                    print(
                        f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}"
                    )

                # New best model, you could save the agent here
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    # Example for saving best model
                    if self.verbose > 0:
                        print(f"Saving new best model to {self.save_path}.zip")
                    self.model.save(self.save_path)

        return True

In [6]:
from stable_baselines3 import PPO
policy_kwargs = dict(
    net_arch=[256, 256]
)
model = PPO("MultiInputPolicy", env, verbose=1, batch_size=256, policy_kwargs=policy_kwargs)
callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=filename)
model.learn(total_timesteps=5e5, progress_bar=True, log_interval=10, callback=callback)

Using cuda device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


Output()

Num timesteps: 1000

Best mean reward: -inf - Last mean reward per episode: 212.60

Saving new best model to tmp/gym/twoarm_ppo/best_model.zip

...

Best mean reward: 309.09 - Last mean reward per episode: 285.94

Num timesteps: 500000

Best mean reward: 309.09 - Last mean reward per episode: 285.73

Num timesteps: 501000

Best mean reward: 309.09 - Last mean reward per episode: 285.35

In [7]:
obs = env.reset()[0]
action_list = []
for i in range(500):
    action, state = model.predict(obs)
    action_list.append(action)
    obs, reward, done,done, info = env.step(action)
    env.render()
    # time.sleep(0.2)
    if done:
        break
env.close()

QObject::moveToThread: Current thread (0x149c2140) is not the object's thread (0x17b0c690).
Cannot move to target thread (0x149c2140)

QObject::moveToThread: Current thread (0x149c2140) is not the object's thread (0x17b0c690).
Cannot move to target thread (0x149c2140)

QObject::moveToThread: Current thread (0x149c2140) is not the object's thread (0x17b0c690).
Cannot move to target thread (0x149c2140)

QObject::moveToThread: Current thread (0x149c2140) is not the object's thread (0x17b0c690).
Cannot move to target thread (0x149c2140)

QObject::moveToThread: Current thread (0x149c2140) is not the object's thread (0x17b0c690).
Cannot move to target thread (0x149c2140)

QObject::moveToThread: Current thread (0x149c2140) is not the object's thread (0x17b0c690).
Cannot move to target thread (0x149c2140)

QObject::moveToThread: Current thread (0x149c2140) is not the object's thread (0x17b0c690).
Cannot move to target thread (0x149c2140)

QObject::moveToThread: Current thread (0x149c2140) is n

In [12]:
import robosuite.macros as macros
macros.IMAGE_CONVENTION = "opencv"
env = GymWrapper(suite.make(
    env_name="TwoArmPegInHole", # try with other tasks like "Stack" and "Door"
    robots=["UR5e", "UR5e"],  # try with other robots like "Sawyer" and "Jaco"
    has_renderer=False,
    ignore_done = True,
    has_offscreen_renderer=True,
    use_object_obs=True,                   # don't provide object observations to agent
    use_camera_obs=True,
    camera_names="frontview",      # use "agentview" camera for observations
    camera_heights=512,                      # image height
    camera_widths=512,                       # image width
    reward_shaping=True,                    # use a dense reward signal for learning
    horizon = 500,
    control_freq=20,                        # control should happen fast enough so that simulation looks smooth
))

In [13]:
import imageio

writer = imageio.get_writer(f'{filename}/video.mp4', fps=20)
obs = env.reset()[0]
for i,v in enumerate(action_list):
    obs, reward, done,done, info = env.step(v)
    frame = obs["frontview_image"]
    writer.append_data(frame)
    print("Saving frame #{}".format(i))
    if done:
        break
writer.close()
env.close()

Saving frame #0
Saving frame #1
Saving frame #2
Saving frame #3
Saving frame #4
Saving frame #5
Saving frame #6
Saving frame #7
Saving frame #8
Saving frame #9
Saving frame #10
Saving frame #11
Saving frame #12
Saving frame #13
Saving frame #14
Saving frame #15
Saving frame #16
Saving frame #17
Saving frame #18
Saving frame #19
Saving frame #20
Saving frame #21
Saving frame #22
Saving frame #23
Saving frame #24
Saving frame #25
Saving frame #26
Saving frame #27
Saving frame #28
Saving frame #29
Saving frame #30
Saving frame #31
Saving frame #32
Saving frame #33
Saving frame #34
Saving frame #35
Saving frame #36
Saving frame #37
Saving frame #38
Saving frame #39
Saving frame #40
Saving frame #41
Saving frame #42
Saving frame #43
Saving frame #44
Saving frame #45
Saving frame #46
Saving frame #47
Saving frame #48
Saving frame #49
Saving frame #50
Saving frame #51
Saving frame #52
Saving frame #53
Saving frame #54
Saving frame #55
Saving frame #56
Saving frame #57
Saving frame #58
Saving 

Saving frame #478
Saving frame #479
Saving frame #480
Saving frame #481
Saving frame #482
Saving frame #483
Saving frame #484
Saving frame #485
Saving frame #486
Saving frame #487
Saving frame #488
Saving frame #489
Saving frame #490
Saving frame #491
Saving frame #492
Saving frame #493
Saving frame #494
Saving frame #495
Saving frame #496
Saving frame #497
Saving frame #498
Saving frame #499
